## Import Libraries

In [1]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.flow.flow import Flow, start, listen  
from langchain_community.tools import DuckDuckGoSearchRun
import textwrap



##  Disable CrewAI's telemetry to prevent data collection and errors

In [2]:
os.environ["CREWAI_TELEMETRY"] = "false"
os.environ["CREWAI_TELEMETRY_DEBUG"] = "0"  

## Load the  LLM


In [3]:
from crewai import LLM


llm = LLM(
    model="groq/qwen/qwen3-32b",
    api_key=os.getenv("GROQ_API_KEY"),
    api_base="https://api.groq.com/openai/v1",
    temperature=0.2,
    max_tokens=1000,
    num_retries=5,
    request_timeout=90
)

## Load  DuckduckGo Search tool

In [4]:
import time
import random
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool("DuckDuckGo Search")
def search_tool1(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    time.sleep(1.0 + random.random()*0.5) 
    return DuckDuckGoSearchRun().run(query)



## Topic for the LLM

In [5]:
topic = "2024 Honda Civic review"

## Defining all Agents

In [6]:
# #  Agent 1: Reviewer A with an Edmunds bias
reviewer1 = Agent(
    role="Reviewer A (Edmunds bias)",
    goal="Search Edmunds for the topic and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Automotive editor who summarizes objectively.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False  

)
#  Agent 2: Reviewer B with a Car and Driver bias
reviewer2 = Agent(
    role="Reviewer B (Car and Driver bias)",
    goal="Search Car and Driver for the topic and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Performance-focused reviewer with balanced takes.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)
# Agent 3: Reviewer C with a Top Gear/other magazine bias
reviewer3 = Agent(
    role="Reviewer C (Top Gear / other mags bias)",
    goal="Search Top Gear or other reputable magazines and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Keeps things practical for buyers.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)

#  Agent 4: Final Synthesizer, to combine the findings
synthesizer = Agent(
    role="Final Synthesizer",
    goal="Merge the three reviewer notes into one clear, non-redundant verdict.Only output the final answer without explaining the steps",
    backstory="Senior editor who reconciles differing opinions.",
    tools=[],  # no web/tools; purely synthesis
    llm=llm,
    allow_delegation=False, 
    verbose=False
)


## Giving tasks descriptions 

In [7]:
# Tasks for the three retrievers
t1 = Task(
    description=f"Topic: {topic}\nSearch site:edmunds.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer1,
    expected_output="5 bullets + verdict + links"
)
t2 = Task(
    description=f"Topic: {topic}\nSearch site:caranddriver.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer2,
    expected_output="5 bullets + verdict + links"
)
t3 = Task(
    description=f"Topic: {topic}\nSearch (site:topgear.com OR site:autocar.co.uk OR site:whatcar.com) {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer3,
    expected_output="5 bullets + verdict + links"
)

from concurrent.futures import ThreadPoolExecutor
from IPython.display import display, Markdown

# Function for running individual tasks in parallel
def run_task_in_parallel(task, agent):
    individual_crew = Crew(
        agents=[agent], 
        tasks=[task], 
        verbose=False
    )
    return individual_crew.kickoff()

# Running t1, t2, t3 in parallel using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=3) as executor:
    future_t1 = executor.submit(run_task_in_parallel, t1, reviewer1)
    future_t2 = executor.submit(run_task_in_parallel, t2, reviewer2) 
    future_t3 = executor.submit(run_task_in_parallel, t3, reviewer3)
    
    # Get results from parallel execution
    result_t1 = future_t1.result()
    result_t2 = future_t2.result()
    result_t3 = future_t3.result()

#  t_final task description
t_final = Task(
    description=(
        f"Based on these three reviews:\n\n"
        f"Review 1 (Edmunds): {result_t1}\n\n"
        f"Review 2 (Car & Driver): {result_t2}\n\n" 
        f"Review 3 (Top Gear): {result_t3}\n\n"
        "Combine the three reviewer outputs into:\n"
        "- 4-6 consensus bullets (no duplicates)\n"
        "- One verdict paragraph (3-5 sentences) for buyers\n"
        "- 'Where reviewers agree' and 'Where they differ' (1-2 bullets each)\n"
        "Keep it neutral, specific, and text-only."
    ),
    agent=synthesizer,
    expected_output="Bullets + verdict paragraph + agreement/differences"
)

# Final synthesis
final_crew = Crew(
    agents=[synthesizer],
    tasks=[t_final], 
    verbose=False
)

final_result = final_crew.kickoff()

# Displaying results
results = [result_t1, result_t2, result_t3, final_result]
agent_roles = ["Reviewer A (Edmunds bias)", "Reviewer B (Car and Driver bias)", "Reviewer C (Top Gear / other mags bias)", "Final Synthesizer"]

for i, (result, role) in enumerate(zip(results, agent_roles)):
    display(Markdown(f"### {i}: {role}\n\n{result}"))

### 0: Reviewer A (Edmunds bias)

- **Pros**: Sporty exterior design, responsive handling, fuel-efficient (up to 32 mpg combined), tech-friendly with 10.2-inch touchscreen and Apple CarPlay, comprehensive safety features.
- **Cons**: Firm ride on rough roads, basic infotainment system, cramped back seat.
- **Verdict**: The 2024 Honda Civic excels as a fun, efficient compact car but sacrifices some comfort for sportiness.
- **Links**: 
  https://www.edmunds.com/2024-honda-civic-review-rating-1954695.html

### 1: Reviewer B (Car and Driver bias)

**2024 Honda Civic Review (Car and Driver):**  
- **Pros:** Punchy 1.5T turbo engine with 182 hp, sharp and confident handling, tech-rich interior with Apple CarPlay/Android Auto, fuel-efficient (32 mpg city/40 mpg highway), quiet cabin at highway speeds.  
- **Cons:** Firm ride quality on base models, limited rear seat legroom, no standard sunroof, 2.0T model’s turbo lag, base model lacks driver-assist features.  
- **Verdict:** A well-rounded compact sedan with agile dynamics and modern tech, though higher trims needed for premium comfort and features.  

**Links:**  
https://www.caranddriver.com/honda/civic/reviews/2024-honda-civic-first-drive-review  
https://www.caranddriver.com/honda/civic/reviews/2024-honda-civic-2-0t-review

### 2: Reviewer C (Top Gear / other mags bias)

- **Pros**: Sharp handling and agile driving dynamics; updated interior with modern tech (Honda's new infotainment system); efficient 1.5L turbo engine with smooth 10-speed automatic; improved ride comfort and reduced road noise; strong safety features (5-star Euro NCAP rating).
- **Cons**: Interior plastics feel slightly cheaper than competitors; limited rear legroom in higher trims; optional tech features can be pricey.
- **Verdict**: The 2024 Honda Civic remains a compelling compact car with a sporty edge, though it lags slightly in interior refinement compared to rivals like the Volkswagen Golf.
- **Links**: 
  https://www.topgear.com/car-reviews/honda-civic-2024-review 
  https://www.autocar.co.uk/car-reviews/honda/civic-2024

### 3: Final Synthesizer

<think>
Okay, let's tackle this. The user wants me to combine three reviews of the 2024 Honda Civic into a concise summary with specific sections. First, I need to extract the key points from each review.

Starting with Edmunds: Pros include sporty design, responsive handling, fuel efficiency (32 mpg), 10.2-inch touchscreen with Apple CarPlay, and safety features. Cons are firm ride, basic infotainment, and cramped back seat. Their verdict highlights efficiency and fun but sacrifices comfort.

Car & Driver mentions the 1.5T engine with 182 hp, sharp handling, tech features, fuel efficiency (32/40 mpg), and a quiet cabin. Cons are firm ride, limited rear legroom, no sunroof, turbo lag in 2.0T, and missing driver-assist in base models. Their verdict says it's well-rounded but needs higher trims for premium features.

Top Gear points out sharp handling, updated interior with new infotainment, efficient engine with 10-speed auto, improved ride comfort, and safety. Cons are cheaper plastics, limited rear legroom in higher trims, and pricey optional tech. Their verdict says it's compelling with a sporty edge but lags in interior refinement compared to Golf.

Now, I need to create 4-6 consensus bullets. Let's see what's common across all three. All mention sporty handling, fuel efficiency (around 32 mpg combined or city), tech features like Apple CarPlay, and rear legroom issues. Also, firm ride quality is a common con. Safety features are mentioned by Edmunds and Top Gear.

For the verdict paragraph, it should encapsulate the main positives and negatives. Emphasize the sporty, efficient, tech features, but note the trade-offs in comfort and interior quality.

Agreement points: All agree on handling, fuel efficiency, tech, rear legroom, and firm ride. Differences: Edmunds and Car & Driver mention base model shortcomings, while Top Gear compares to Golf. Also, Edmunds and Car & Driver note specific engine issues (turbo lag in 2.0T, base model lacks features), while Top Gear talks about interior plastics and optional tech pricing.

Need to make sure bullets are concise and non-redundant. Check for duplicates. For example, fuel efficiency is mentioned in all three, so that's a consensus. Handling is another. Rear legroom is a common con. Tech features like Apple CarPlay are across all. Safety features are in two reviews but maybe not all three, but since two mention it, maybe include as consensus. Wait, Edmunds and Top Gear mention safety, Car & Driver mentions driver-assist features as missing in base models. So safety features are present but maybe not standard across all trims. Hmm. Maybe consensus on comprehensive safety features.

Need to verify each point. Let me list all the pros and cons from each review and cross-reference.

Pros Consensus:
- Sporty handling/ agile driving dynamics (all three)
- Fuel-efficient (32 mpg combined or similar)
- Tech features (Apple CarPlay, touchscreen)
- Safety features (Edmunds, Top Gear)
- Improved ride comfort (Top Gear mentions improved, but others mention firm ride. Wait, that's conflicting. Wait, Top Gear says improved ride comfort, but Edmunds and Car & Driver mention firm ride. So maybe not consensus. Hmm. Wait, the consensus bullets need to be points all agree on. So maybe not ride comfort. Let me check again.

Wait, Edmunds says firm ride on rough roads, Car & Driver says firm ride quality on base models, Top Gear says improved ride comfort. So actually, they differ on ride quality. So that's a difference. So consensus bullets can't include ride comfort. So consensus pros are handling, fuel efficiency, tech features, and rear legroom issues as a con. Wait, all three mention rear legroom as a con. So that's a consensus con.

So consensus bullets:

Pros:
- Sporty handling and agile driving dynamics
- Fuel-efficient (32 mpg combined or similar)
- Tech-friendly with Apple CarPlay and modern infotainment
- Comprehensive safety features (Edmunds and Top Gear mention, Car & Driver mentions driver-assist features missing in base models but others have them)

Cons:
- Limited rear seat legroom
- Base models lack some features (driver-assist, sunroof)
- Interior plastics feel cheaper in some trims (Top Gear)

Wait, but Car & Driver mentions no standard sunroof and base models lack driver-assist. Top Gear mentions interior plastics. Edmunds says basic infotainment. So maybe the consensus con is limited rear legroom and base model features. But Edmunds says cramped back seat, Car & Driver limited rear legroom, Top Gear limited rear legroom in